In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
Employees = [
    {"id_no": 101, "first_name": "Anata", "last_name": "Sarkar", "age": 28, "salary": 55000.00},
    {"id_no": 102, "first_name": "Asutosh", "last_name": "Mukherjee", "age": 34, "salary": 63000.00},
    {"id_no": 103, "first_name": "Rada", "last_name": "Sengupto", "age": 25, "salary": 45000.00},
    {"id_no": 104, "first_name": "Bulai", "last_name": "Chandra", "age": 45, "salary": 80000.00},
    {"id_no": 105, "first_name": "Chaitali", "last_name": "Mukherjee", "age": 30, "salary": 57000.00}
]
#employee with highest salary and age > 30
highest_salary_employee = None
max_salary = -1
#employee with age > 30 and highest salary
for employee in employees:
    if employee['age'] > 30:
        if employee['salary'] > max_salary:
            max_salary = employee['salary']
            highest_salary_employee = employee
if highest_salary_employee:
    print(f"The employee with age > 30 and highest salary is: {highest_salary_employee['first_name']} {highest_salary_employee['last_name']} with salary {highest_salary_employee['salary']}")
#using POS tagging for the result of the Query
    full_name = f"{highest_salary_employee['first_name']} {highest_salary_employee['last_name']}"
    doc = nlp(full_name)
    print(f"POS tags for {full_name}:")
    for token in doc:
        print(f"{token.text}: {token.pos_}")
else:
    print("No employee found with age > 30.")


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
Employees = [
    {"id_no": 101, "first_name": "Anata", "last_name": "Sarkar", "age": 28, "salary": 55000.00},
    {"id_no": 102, "first_name": "Asutosh", "last_name": "Mukherjee", "age": 34, "salary": 63000.00},
    {"id_no": 103, "first_name": "Rada", "last_name": "Sengupto", "age": 25, "salary": 45000.00},
    {"id_no": 104, "first_name": "Bulai", "last_name": "Chandra", "age": 45, "salary": 80000.00},
    {"id_no": 105, "first_name": "Chaitali", "last_name": "Mukherjee", "age": 30, "salary": 57000.00}
]
#employee with highest salary and age > 30
highest_salary_employee = None
max_salary = -1
for employee in employees:
    if employee['age'] > 30:
        if employee['salary'] > max_salary:
            max_salary = employee['salary']
            highest_salary_employee = employee
if highest_salary_employee:
    employee_info = f"The employee with age > 30 and highest salary is: {highest_salary_employee['first_name']} {highest_salary_employee['last_name']} with salary {highest_salary_employee['salary']}"
    print(employee_info)
#POS tagging to the employee_info string itself
    doc = nlp(employee_info)
    print("POS tags for the employee information:")
    for token in doc:
        print(f"{token.text}: {token.pos_}")
else:
    print("No employee found with age > 30.")


In [3]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Sample question
question = "I want to find the employee whose age is above 30 and salary is highest"

# Parse the question using spaCy
doc = nlp(question)

# Function to extract table name from the parsed tokens
def extract_table_name(doc):
    for token in doc:
        if token.dep_ == "dobj":  # direct object, which is often the table name
            return token.text
    return None

# Function to extract conditions from the parsed tokens
def extract_conditions(doc):
    conditions = []
    for token in doc:
        if token.dep_ == "prep" and token.head.text == "age":
            conditions.append(f"age {token.head.head.text} {token.nbor().text}")
        elif token.dep_ == "amod" and token.head.text == "salary":
            conditions.append("salary = (SELECT MAX(salary) FROM employee)")
    return conditions

# Function to convert parsed tokens into SQL query components
def parse_question_to_sql(doc):
    table_name = extract_table_name(doc)
    conditions = extract_conditions(doc)

    # Construct the SQL query
    if table_name:
        sql_query = f"SELECT * FROM {table_name}"
        if conditions:
            sql_query += " WHERE " + " AND ".join(conditions)
        return sql_query
    else:
        return "Table name not found in the query"

# Convert the parsed question to an SQL query
sql_query = parse_question_to_sql(doc)
print(sql_query)


SELECT * FROM employee


In [ ]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Sample sentence
sentence = "Find employees aged above 30 with the highest salary."

# Function to convert a sentence to SQL command using POS tagging
def sentence_to_sql_with_pos(sentence):
    doc = nlp(sentence)

    # Initialize variables for constructing the SQL query
    table_name = "Employees"
    select_clause = "SELECT *"
    where_clause = ""
    order_by_clause = ""
    limit_clause = "FETCH FIRST 1 ROWS ONLY"

    # Flags to track if age and salary conditions are found
    age_condition_found = False
    salary_condition_found = False

    # Extract components based on POS tagging
    for token in doc:
        if token.pos_ == "NOUN" and token.text.lower() == "employees":
            continue  # Skip reference to table name
        if token.pos_ == "VERB" and token.text.lower() == "find":
            continue  # Skip verbs not relevant to SQL construction
        if token.pos_ == "ADJ" and token.head.pos_ == "NOUN":
            if token.head.text.lower() == "age" and token.text.lower() in ("above", "greater", "more"):
                next_token = doc[token.i + 1]
                where_clause += f"age > {next_token.text} "
                age_condition_found = True
            elif token.head.text.lower() == "salary" and token.text.lower() == "highest":
                order_by_clause = "ORDER BY salary DESC "
                salary_condition_found = True

    # If age condition is not found, default to age > 30
    if not age_condition_found:
        where_clause = "WHERE age > 30 "

    # Construct the final SQL query
    sql_query = f"{select_clause} FROM {table_name} "
    if where_clause:
        sql_query += where_clause
    if order_by_clause:
        sql_query += order_by_clause
    sql_query += limit_clause

    return sql_query.strip()

# Convert the sentence to SQL command using POS tagging
sql_command = sentence_to_sql_with_pos(sentence)
print("Generated SQL command:")
print(sql_command)


In [ ]:
import spacy

# Sample Employees database
Employees = [
    {"id_no": 101, "first_name": "Anata", "last_name": "Sarkar", "age": 28, "salary": 55000.00},
    {"id_no": 102, "first_name": "Asutosh", "last_name": "Mukherjee", "age": 34, "salary": 63000.00},
    {"id_no": 103, "first_name": "Rada", "last_name": "Sengupto", "age": 25, "salary": 45000.00},
    {"id_no": 104, "first_name": "Bulai", "last_name": "Chandra", "age": 45, "salary": 80000.00},
    {"id_no": 105, "first_name": "Chaitali", "last_name": "Mukherjee", "age": 30, "salary": 57000.00}
]

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to convert a sentence to SQL command using POS tagging
def sentence_to_sql(sentence, database):
    doc = nlp(sentence)

    # Initialize variables for constructing the SQL query
    table_name = "Employees"
    select_clause = "SELECT *"
    where_conditions = []
    order_by_clause = ""
    limit_clause = "FETCH FIRST 1 ROW ONLY"  # Adjusted for Oracle syntax

    # Mapping of key phrases to SQL conditions
    key_phrases = {
        "employee": {
            "name": "first_name || ' ' || last_name",
            "age": "age",
            "salary": "salary"
        }
    }

    # Extract components based on POS tagging
    for token in doc:
        if token.pos_ == "NOUN" and token.text.lower() == "employees":
            continue  # Skip reference to table name
        if token.pos_ == "VERB" and token.text.lower() in ("want", "need", "find", "show", "get", "give"):
            continue  # Skip verbs not relevant to SQL construction

        # Handle specific conditions based on POS tagging
        if token.pos_ == "NOUN" and token.text.lower() in key_phrases:
            entity = key_phrases[token.text.lower()]
            for child in token.children:
                if child.pos_ == "PROPN" and child.text.lower() in entity:
                    condition_key = entity[child.text.lower()]
                    if condition_key == "name":
                        names = child.text.split()
                        if len(names) == 2:
                            # Adjusting to handle first name and last name separately
                            first_name = names[0]
                            last_name = names[1]
                            where_conditions.append(f"(first_name = '{first_name}' AND last_name = '{last_name}') OR (first_name = '{last_name}' AND last_name = '{first_name}')")

    # Construct the final SQL query
    sql_query = f"{select_clause} FROM {table_name}"
    if where_conditions:
        sql_query += f" WHERE {' AND '.join(where_conditions)}"
    sql_query += f" {limit_clause}"

    return sql_query.strip()

# Main function to prompt user input and generate SQL query
def main():
    print("Enter an English sentence to generate a SQL query:")
    user_input = input("> ")

    # Generate SQL query based on user input
    sql_command = sentence_to_sql(user_input, Employees)
    print("\nGenerated SQL command:")
    print(sql_command)

# Run the main function
if __name__ == "__main__":
    main()


Enter an English sentence to generate a SQL query:


In [ ]:
import spacy

# Sample Employees database
Employees = [
    {"id_no": 101, "first_name": "Anata", "last_name": "Sarkar", "age": 28, "salary": 55000.00},
    {"id_no": 102, "first_name": "Asutosh", "last_name": "Mukherjee", "age": 34, "salary": 63000.00},
    {"id_no": 103, "first_name": "Rada", "last_name": "Sengupto", "age": 25, "salary": 45000.00},
    {"id_no": 104, "first_name": "Bulai", "last_name": "Chandra", "age": 45, "salary": 80000.00},
    {"id_no": 105, "first_name": "Chaitali", "last_name": "Mukherjee", "age": 30, "salary": 57000.00}
]

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to convert a sentence to SQL command using POS tagging
def sentence_to_sql(sentence, database):
    doc = nlp(sentence)

    # Initialize variables for constructing the SQL query
    table_name = "Employees"
    select_clause = "SELECT *"
    where_conditions = []
    order_by_clause = ""
    limit_clause = "FETCH FIRST 1 ROW ONLY"  # Adjusted for Oracle syntax

    # Mapping of key phrases to SQL conditions
    key_phrases = {
        "employee": {
            "name": "first_name || ' ' || last_name",
            "age": "age",
            "salary": "salary"
        }
    }

    # Extract components based on POS tagging
    for token in doc:
        if token.pos_ == "NOUN" and token.text.lower() == "employees":
            continue  # Skip reference to table name
        if token.pos_ == "VERB" and token.text.lower() in ("want", "need", "find", "show", "get", "give"):
            continue  # Skip verbs not relevant to SQL construction

        # Handle specific conditions based on POS tagging
        if token.pos_ == "NOUN" and token.text.lower() in key_phrases:
            entity = key_phrases[token.text.lower()]
            for child in token.children:
                if child.pos_ == "PROPN" and child.text.lower() in entity:
                    condition_key = entity[child.text.lower()]
                    if condition_key == "name":
                        names = child.text.split()
                        if len(names) == 2:
                            # Adjusting to handle first name and last name separately
                            first_name = names[0]
                            last_name = names[1]
                            where_conditions.append(f"(first_name = '{first_name}' AND last_name = '{last_name}') OR (first_name = '{last_name}' AND last_name = '{first_name}')")

    # Construct the final SQL query
    sql_query = f"{select_clause} FROM {table_name}"
    if where_conditions:
        sql_query += f" WHERE {' AND '.join(where_conditions)}"
    sql_query += f" {limit_clause}"

    return sql_query.strip()

# Main function to prompt user input and generate SQL query
def main():
    print("Enter an English sentence to generate a SQL query:")
    user_input = input("> ")

    # Generate SQL query based on user input
    sql_command = sentence_to_sql(user_input, Employees)
    print("\nGenerated SQL command:")
    print(sql_command)

# Run the main function
if __name__ == "__main__":
    main()


In [28]:
import spacy

# Sample Employees database
Employees = [
    {"id_no": 101, "first_name": "Anata", "last_name": "Sarkar", "age": 28, "salary": 55000.00},
    {"id_no": 102, "first_name": "Asutosh", "last_name": "Mukherjee", "age": 34, "salary": 63000.00},
    {"id_no": 103, "first_name": "Rada", "last_name": "Sengupto", "age": 25, "salary": 45000.00},
    {"id_no": 104, "first_name": "Bulai", "last_name": "Chandra", "age": 45, "salary": 80000.00},
    {"id_no": 105, "first_name": "Chaitali", "last_name": "Mukherjee", "age": 30, "salary": 57000.00}
]

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to convert a sentence to SQL command using POS tagging
def sentence_to_sql(sentence, database):
    doc = nlp(sentence)

    # Initialize variables for constructing the SQL query
    table_name = "Employees"
    select_clause = "SELECT *"
    where_conditions = []
    order_by_clause = ""
    limit_clause = "FETCH FIRST 1 ROW ONLY"  # Adjusted for Oracle syntax

    # Mapping of key phrases to SQL conditions
    key_phrases = {
        "employee": {
            "name": "first_name || ' ' || last_name",
            "age": "age",
            "salary": "salary"
        }
    }

    # Extract components based on POS tagging
    for token in doc:
        if token.pos_ == "NOUN" and token.text.lower() == "employees":
            continue  # Skip reference to table name
        if token.pos_ == "VERB" and token.text.lower() in ("want", "need", "find", "show", "get", "give"):
            continue  # Skip verbs not relevant to SQL construction

        # Handle specific conditions based on POS tagging
        if token.pos_ == "NOUN" and token.text.lower() in key_phrases:
            entity = key_phrases[token.text.lower()]
            for child in token.children:
                if child.pos_ == "PROPN" and child.text.lower() in entity:
                    condition_key = entity[child.text.lower()]
                    if condition_key == "name":
                        names = child.text.split()
                        if len(names) == 2:
                            # Adjusting to handle first name and last name separately
                            first_name = names[0]
                            last_name = names[1]
                            where_conditions.append(f"(first_name = '{first_name}' AND last_name = '{last_name}') OR (first_name = '{last_name}' AND last_name = '{first_name}')")

    # Construct the final SQL query
    sql_query = f"{select_clause} FROM {table_name}"
    if where_conditions:
        sql_query += f" WHERE {' AND '.join(where_conditions)}"
    sql_query += f" {limit_clause}"

    return sql_query.strip()

# Main function to prompt user input and generate SQL query
def main():
    print("Enter an English sentence to generate a SQL query:")
    user_input = input("> ")

    # Generate SQL query based on user input
    sql_command = sentence_to_sql(user_input, Employees)
    print("\nGenerated SQL command:")
    print(sql_command)

# Run the main function
if __name__ == "__main__":
    main()


Enter an English sentence to generate a SQL query:


>  employees with salaries more than 50000



Generated SQL command:
SELECT * FROM Employees FETCH FIRST 1 ROW ONLY
